In [1]:
import pandas as pd

In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
import joblib
import plotly.express as px

In [4]:
data = pd.read_csv('03_data_procesada/01_data_pro_1.csv')

categorical_cols = ['BIOPSIA', 'ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS']
numerical_cols = [col for col in data.columns if col not in categorical_cols + ['HOSPITALIZACION']]

In [5]:
hospitalizacion_counts = data['HOSPITALIZACION'].value_counts().reset_index()
hospitalizacion_counts.columns = ['HOSPITALIZACION', 'count']

fig = px.bar(hospitalizacion_counts, x='HOSPITALIZACION', y='count', 
             title='Distribución de la Variable HOSPITALIZACION',
             labels={'HOSPITALIZACION': 'Hospitalización', 'count': 'Cantidad'},
             text='count')
fig.show()

Como no se aplicara ninguna Tecnica de Sobre muestreo cobra mucha importancia la Stratificacion de la Variable Target

In [6]:
X = data.drop('HOSPITALIZACION', axis=1)
y = data['HOSPITALIZACION']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

In [7]:
categorical_transformer = Pipeline(steps=[
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore'))])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('clf', xgb.XGBClassifier())
])

Es evidente que en la variable `param_grid` se corrió el modelo con muchos valores distintos en los Hiperparámetros y al final estoy dejando solo los valores con los que se comporta mejor el modelo

In [8]:
param_grid = {
    'clf__max_depth': [3],
    'clf__learning_rate': [1.30],
    'clf__n_estimators': [5]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

# MODELO
best_model = grid_search.best_estimator_

# EVALUACION CON EL CONJUNTO DE PRUEBA
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# MEJOR COMBINACION DE HIPERPARAMETROS
best_params = grid_search.best_params_
print(f"Mejor combinación de hiperparámetros: {best_params}")

# EVALUACION CONJUNTO DE TEST
print('*'*50)
y_pred_train = best_model.predict(X_train)
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       259
           1       1.00      0.77      0.87        13

    accuracy                           0.99       272
   macro avg       0.99      0.88      0.93       272
weighted avg       0.99      0.99      0.99       272

[[259   0]
 [  3  10]]
Mejor combinación de hiperparámetros: {'clf__learning_rate': 1.3, 'clf__max_depth': 3, 'clf__n_estimators': 5}
**************************************************
[[259   0]
 [  3  10]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       259
           1       1.00      0.77      0.87        13

    accuracy                           0.99       272
   macro avg       0.99      0.88      0.93       272
weighted avg       0.99      0.99      0.99       272



*** 
Medicion a Pie

In [9]:
y_prueba = best_model.predict(X)

resultados = pd.DataFrame()
resultados['y_real'] = y
resultados['y_prueba'] = y_prueba
resultados['diferencia'] = resultados['y_real'] - resultados['y_prueba']
resultados['diferencia'].value_counts()

matriz = confusion_matrix(y, y_prueba)


tn, fp, fn, tp = matriz.ravel()

sensibilidad =(tp / (tp + fn))
especificidad =(tn / (tn + fp))
precision =(tp / (tp + fp))
exactitud =((tp + tn) / (tp + tn + fp + fn) )

print(f'Sensibilidad:   {sensibilidad}')
print(f'Especificidad:  {especificidad}')
print(f'Precision:      {precision}')
print(f'Exactitud:      {exactitud}')

Sensibilidad:   0.7692307692307693
Especificidad:  1.0
Precision:      1.0
Exactitud:      0.9889705882352942


In [ ]:
resultados

***
Guardando el Modelo

In [10]:
import joblib

In [ ]:
joblib.dump(best_model, 'models/modelo_proyecto_01.pkl')

***
Cargando el Modelo

In [ ]:
modelo_cargado = joblib.load('models/modelo_proyecto_01.pkl')

Como se Probaria el Modelo en la App

In [ ]:
import pandas as pd

In [ ]:
predictors = ['EDAD', 'DIABETES', 'HOSPITALIZACIÓN ULTIMO MES', 'PSA', 'BIOPSIAS PREVIAS', 'VOLUMEN PROSTATICO',
              'ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS', 'NUMERO DE MUESTRAS TOMADAS', 'BIOPSIA', 'FIEBRE']

inputs = {
    'EDAD': 70, 
    'DIABETES': 0, 
    'HOSPITALIZACIÓN ULTIMO MES': 0, 
    'PSA': 18.0, 
    'BIOPSIAS PREVIAS': 0, 
    'VOLUMEN PROSTATICO': 1,
    'ANTIBIOTICO UTILIAZADO EN LA PROFILAXIS': 'FLUOROQUINOLONA_AMINOGLICOSIDO', 
    'NUMERO DE MUESTRAS TOMADAS': 12, 
    'BIOPSIA': 'NEG', 
    'FIEBRE': 0
}
valores_test = pd.DataFrame(inputs, index=[0])

prediccion_input = modelo_cargado.predict(valores_test)
prediccion_input[0]

Valores que pueden Tomar las Variables

In [ ]:
edades = [39,84]                      #Valor Minimo y Valor Maximo
diabetes = [0,1]                      #Variable de Si o No  1 o 0
hospitalizacion_ultimo_mes = [0,1]    #Variable de Si o No  1 o 0  
psa = [0.6,100]                       #Valor Minimo y Valor Maximo
biopsias_previas = [0,1]              #Variable de Si o No  1 o 0
volumen_prostatico = [0,1]            #Variable de Si o No  1 o 0 
antibiotico_utilizado = ['FLUOROQUINOLONA_AMINOGLICOSIDO', 'CEFALOSPORINA_AMINOGLUCOCIDO',
                          'OROQUINOLONAS', 'FLUOROQUINOLONA_AMINOGLICÓSIDO', 'OTROS']
numero_muestras = [4,24]              #Valor Minimo y Valor Maximo
biopsia = ['NEG', 'ADENOCARCINOMA GLEASON 6 ', 'ADENOCARCINOMA GLEASON 7 ', 'ADENOCARCINOMA GLEASON 6', 
           'ADENOCARCINOMA GLEASON 10 ', 'ADENOCARCINOMA GLEASON 9 ', 'ADENOCARCINOMA GLEASON 8 ',
           'PROSTATITIS', 'ADENOCARCINOMA GLEASON 7', 'HIPERPLASIA PROSTATICA', 'CARCINOMA INDIFERENCIADO DE CELULAS CLARAS']
fiebre = [0,1]                        #Variable de Si o No  1 o 0


***
Preparando la Data de la Tabla de la app

In [ ]:
mask = data['HOSPITALIZACION'] == 1
hospitalizados = data.loc[mask]

mask = data['HOSPITALIZACION'] == 0
no_hospitalizados = data.loc[mask].sample(n=26)

In [ ]:
tabla = pd.concat([hospitalizados, no_hospitalizados], axis=0)

In [ ]:
tabla.to_csv('04_data_ejemplo_tabla_app/tabla_proj_1.csv', index=False)